In [1]:
!pip install yfinance --upgrade --no-cache-dir
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

  Found existing installation: yfinance 0.1.43
    Uninstalling yfinance-0.1.43:
      Successfully uninstalled yfinance-0.1.43
  Running setup.py install for yfinance: started
    Running setup.py install for yfinance: finished with status 'done'


# 1a. Download 2 year stock prices of Apple and Facebook in 2018 2019

In [3]:
ticker = ['AAPL','FB']
df = yf.download(ticker, start='2018-01-01', end='2019-12-31', progress=False)
df = df.replace("", float("NaN"))
df = df.dropna()
df.head()

Adj Close                   Close                    High  \
                  AAPL          FB        AAPL          FB        AAPL   
Date                                                                     
2018-01-02  166.804016  181.419998  172.259995  181.419998  172.300003   
2018-01-03  166.774963  184.669998  172.229996  184.669998  174.550003   
2018-01-04  167.549622  184.330002  173.029999  184.330002  173.470001   
2018-01-05  169.457214  186.850006  175.000000  186.850006  175.369995   
2018-01-08  168.827820  188.279999  174.350006  188.279999  175.610001   

                               Low                    Open              \
                    FB        AAPL          FB        AAPL          FB   
Date                                                                     
2018-01-02  181.580002  169.259995  177.550003  170.160004  177.679993   
2018-01-03  184.779999  171.960007  181.330002  172.529999  181.880005   
2018-01-04  186.210007  172.080002  184.100006  172.539993  184.899994   
2018-01-05  186.899994  173.050003  184.929993  173.440002  185.589996   
2018-01-08  188.899994  173.929993  186.330002  174.350006  187.199997   

              Volume            
                AAPL        FB  
Date                            
2018-01-02  25555900  18151900  
2018-01-03  29517900  16886600  
2018-01-04  22434600  13880900  
2018-01-05  23660000  13574500  
2018-01-08  20567800  17994700

# 1b. Compute log return from 'Adj Close' stock price

In [4]:
stock1_logreturn = np.log(df['Adj Close'][ticker[0]]) - np.log(df['Adj Close'][ticker[0]].shift(1))
stock1_logreturn = stock1_logreturn.replace(0, float("NaN"))
stock1_logreturn = stock1_logreturn.dropna()

stock2_logreturn = np.log(df['Adj Close'][ticker[1]]) - np.log(df['Adj Close'][ticker[1]].shift(1))
stock2_logreturn = stock2_logreturn.replace(0, float("NaN"))
stock2_logreturn = stock2_logreturn.dropna()

# 1c. Compute the mean, standard deviation, skewness, excess kurtosis of their log returns

# 1d. Repeat for both stocks

In [5]:
stock1_logreturn_mean = stock1_logreturn.mean()
stock1_logreturn_std = stock1_logreturn.std()
stock1_logreturn_skew = stock1_logreturn.skew()
stock1_logreturn_kurtosis = stock1_logreturn.kurtosis()
print(f"{ticker[0]} log-return mean: {stock1_logreturn_mean}")
print(f"{ticker[0]} log-return standard deviation: {stock1_logreturn_std}")
print(f"{ticker[0]} log-return skewness: {stock1_logreturn_skew}")
print(f"{ticker[0]} log-return excess kurtosis: {stock1_logreturn_kurtosis}")
print()

stock2_logreturn_mean = stock2_logreturn.mean()
stock2_logreturn_std = stock2_logreturn.std()
stock2_logreturn_skew = stock2_logreturn.skew()
stock2_logreturn_kurtosis = stock2_logreturn.kurtosis()
print(f"{ticker[1]} log return mean: {stock2_logreturn_mean}")
print(f"{ticker[1]} log return standard deviation: {stock2_logreturn_std}")
print(f"{ticker[1]} log return skewness: {stock2_logreturn_skew}")
print(f"{ticker[1]} log return excess kurtosis: {stock2_logreturn_kurtosis}")

AAPL log-return mean: 0.0011096178767915736
AAPL log-return standard deviation: 0.01741569778107351
AAPL log-return skewness: -0.5864202028626524
AAPL log-return excess kurtosis: 4.339130586306103

FB log return mean: 0.0002391042557785599
FB log return standard deviation: 0.021440506052068688
FB log return skewness: -1.850757057850761
FB log return excess kurtosis: 20.33606009364708


# 1e. 
## Question 1: Compute covariance and correlation.

In [6]:
df_logreturn = pd.concat([stock1_logreturn, stock2_logreturn], axis=1)
stock1_stock2_covariance = df_logreturn.cov()
stock1_stock2_correlation = df_logreturn.corr()
print(f"{ticker[0]} - {ticker[1]} log return covariance")
print(stock1_stock2_covariance)
print()
print(f"{ticker[0]} - {ticker[1]} log return correlation")
print(stock1_stock2_correlation)

AAPL - FB log return covariance
          AAPL        FB
AAPL  0.000303  0.000168
FB    0.000168  0.000460

AAPL - FB log return correlation
          AAPL        FB
AAPL  1.000000  0.450012
FB    0.450012  1.000000


## Question 2: Explain their difference. How do you convert one to the other?
### Covariance measures how two variables move with respect to each other. Correlation is a step ahead of covariance as it quantifies the relationship between two random variables, it's a unit measure of how these variables change with respect to each other (normalized covariance value)
#### The Cov(AAPL, FB) = 0.000168 > 0 indicates that those stock log returns tend to move together. Cross check with Corr(AAPL, FB) = 0.450063 is the magnitude of the measurement how these two stocks change with respect to each other

### Covariance = Correlation * Standard Deviation of Stock A * Standard Deviation of Stock B
#### Cov(AAPL, FB) = Corr(AAPL, FB) * Std(AAPL) * Std(FB) = 0.450063*0.017415*0.021440 = 0.000168

# 2.a Divide data into 2 parts: train dataset 80% data and test data set 20% data
### Test data set is the latest data 
### Train data set is older than test data set

In [7]:
df_train, df_test = train_test_split(df_logreturn, test_size=0.2, shuffle=False)
df_train = df_train.dropna()
df_test = df_test.dropna()

# 2.b This function to categorize each day in 2 year prices history as beloging to one of four category:
1. Both stocks up
2. Stock \#1 up, stock \#2 down
3. Stock \#1 down, stock \#2 up
4. Both stocks down

# 2.c Build transition matrix of portfolio direction

In [9]:
def transition_matrix(df_input):
    df_input['movement'] = ''
    df_input = df_input.dropna()

    STATE_UP_UP = 1
    STATE_DOWN_DOWN = -1
    STATE_UP_DOWN = 0.5
    STATE_DOWN_UP = -0.5

    for index, row in df_input.iterrows():
        if row[ticker[0]] > 0 and row[ticker[1]] > 0:
            df_input.loc[index, ['movement']] = STATE_UP_UP
        elif row[ticker[0]] < 0 and row[ticker[1]] < 0:
            df_input.loc[index, ['movement']] = STATE_DOWN_DOWN
        elif row[ticker[0]] > 0 and row[ticker[1]] < 0:
            df_input.loc[index, ['movement']] = STATE_UP_DOWN
        elif row[ticker[0]] < 0 and row[ticker[1]] > 0:
            df_input.loc[index, ['movement']] = STATE_DOWN_UP

    df_input['movement_tomorrow'] = df_input['movement'].shift(-1)
    together_together = 0
    together_apart = 0
    apart_together = 0
    apart_apart = 0

    for index, row in df_input.iterrows():
        test = row['movement'] + row['movement_tomorrow']
        # together then together
        if test == 2 or test == -2:
            together_together+=1
        # apart then apart
        elif test == 1 or test == -1:
            apart_apart+=1
        # distinguish together_together and apart_apart
        elif test == 0:
            if np.abs(row['movement']) == 1:
                together_together += 1 
            else:
                apart_apart += 1
        # apart_together and together_apart
        elif np.abs(test) == 0.5 or np.abs(test) == 1.5:
            if np.abs(row['movement']) == 1:
                together_apart +=1
            else:
                apart_together += 1

    columns = {
        'together':[together_together/(together_together+together_apart), apart_together/(apart_together+apart_apart)],
        'apart':[together_apart/(together_together+together_apart), apart_apart/(apart_together+apart_apart)],
    }
    transition_matrix = pd.DataFrame(columns, index=['together', 'apart'])

    return transition_matrix

In [10]:
# Transition matrix of test data set 20% data
test_transition_matrix = transition_matrix(df_test)
test_transition_matrix.head()

,together,apart
together,0.676471,0.323529
apart,0.656250,0.343750


In [11]:
# Transition matrix of train data set 80% data
train_transition_matrix = transition_matrix(df_train)
train_transition_matrix.head()

,together,apart
together,0.665370,0.334630
apart,0.621429,0.378571


# 2.d
### Both train and test data set shows the dominance probability of the portfolio moves from together to together, apart to together compares to other 2 scenarios that the portfolio moves from apart to apart and together to apart.
### The 20% latest data (test data set) illustrates the increase in scenarios moving together to together, apart to together. Which means the portoflio recently tends to more moving according to those 2 scenarios.
### The 80% older data (train data set) has more probability regarding the scenarios that the portfolio moves from together to apart, apart to apart. Which means the portoflio recently tends to less moving according to those 2 scenarios.

# 2.e Is the process Markovian?
### This process is Markovian, because it's memory-less. This process estimates the probability that the portfolio return moves from yesterday state to today state without considering the historical data